# **Adaptive RAG**
すべてのクエリを同じ方法で処理するのではなく、各クエリタイプに最適な戦略を選択します。

具体的にはシステムがクエリを分析し、次の２つの内、最適な処理方法を決定します。
- Self-corrective RAG：インデックス化されたデータを使用して回答できる質問の場合。
- ウェブ検索：インデックスにない情報を必要とする質問の場合。

## Self-corrective RAG とは？

Self-corrective RAG とは、自己修正機構を組み込んだ手法です。通常のRAGでは、外部データから取得した情報をもとに初期の回答を生成しますが、Self-corrective RAG では以下のプロセスが追加されます：

1. **初期生成**  
   モデルが外部情報を利用して初期の回答を生成します。

2. **自己評価と誤り検出**  
   生成された回答をモデル自身(self)が評価し、誤りや不十分な点を検出します。

3. **再取得と修正**  
   検出された問題を補うため、再度情報検索を行い、取得した情報を基に回答を修正・改善します。

```mermaid
%%{init: {'flowchart': {'curve': 'linear'}}}%%
graph TD;
    Start([<p>__start__</p>]):::first
    WebSearch[web_search]
    Retrieve[retrieve]
    GradeDocs[grade_documents]
    Generate[generate]
    End([<p>__end__</p>]):::last

    %% ルーティング（START から）
    Start -.->|vectorstore| Retrieve
    Start -.->|web_search| WebSearch

    %% 情報取得と処理
    WebSearch --> Generate
    Retrieve --> GradeDocs
    GradeDocs -->|generate| Generate

    %% 生成結果の評価・再試行制御（Generate ノード内の自己ループと終了）
    Generate -.->|not supported| Generate
    Generate -.->|not useful| Generate
    Generate -.->|max_retries_reached| End
    Generate -.->|useful| End

    classDef first fill:#fff,stroke:#333,stroke-width:2px;
    classDef last fill:#bfb6fc,stroke:#333,stroke-width:2px;
```

# 評価用データの準備

In [73]:
# from datasets import load_dataset
# ds = load_dataset("allganize/RAG-Evaluation-Dataset-JA")

In [74]:
# import pandas as pd
# df = ds['test'].to_pandas()
# df.head(3)

In [75]:
# eval_df = df[df['target_file_name'] == 'kaisetsushiryou_2024.pdf']
# eval_df = eval_df[['question', 'target_answer', 'target_page_no']]
# eval_df.to_csv('../data/eval_data.csv', index=False)

import pandas as pd
eval_df = pd.read_csv('../data/eval_data.csv')
eval_df.head(3)

,question,target_answer,target_page_no
0,オープンイノベーション促進税制において、スタートアップ企業の株式取得に対する税制優遇措置は、...,オープンイノベーション促進税制の下で、新規発行株式の取得は「新規出資型」として分類され、発行...,13
1,イノベーション拠点税制における所得控除について、控除対象となる研究開発活動に関して具体的にど...,イノベーション拠点税制における所得控除の対象となるためには、企業が主に「国内で」「自ら」開発...,16
2,「カーブアウト加速等支援事業」の主な目的は何ですか？,事業会社に蓄積されている技術を活用し、新たな会社を立ち上げた者や立ち上げる意思を持つ者に研究...,21


In [76]:
print(len(eval_df))

7


In [77]:
# !uv pip install -qU langchain-community "langchain[aws]" boto3

# API KEYの準備

In [78]:
import os
from dotenv import load_dotenv
load_dotenv()

# # for web search
os.environ['TAVILY_API_KEY'] = os.getenv('TAVILY_API_KEY')
# # for llm
os.environ["OPENAI_API_KEY"]  = os.getenv('OPENAI_API_KEY') # 一回の実行で1000円くらい取られます。

In [79]:
# huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks... To disable this warning, you can either: - Avoid using tokenizers before the fork if possible - Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
# os.environ["TOKENIZERS_PARALLELISM"] = "true" # 警告対策　tokenizersライブラリの並列処理を明示的にON 

# load embedding model
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(model="text-embedding-3-small") # デフォルトと同じだが明示的にした
# 次元数: 1536
# コンテキスト長: 8191トークン
# スループット: 非常に高速

# # load data
# from langchain_community.document_loaders import PyPDFLoader
# loader = PyPDFLoader("../data/pdf/57_public_スタートアップ育成に向けた政府の取組_file_name=kaisetsushiryou_2024.pdf")
# documents = loader.load()

# # split documents
# from langchain.text_splitter import RecursiveCharacterTextSplitter
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
# documents = text_splitter.split_documents(documents)

In [80]:
# from langchain.vectorstores import FAISS

# # Create vectorstore
# vectorstore = FAISS.from_documents(documents, embeddings)

# # Save locally
# vectorstore.save_local("../data/db/faiss_index")

# 先に作っておいたデータベースのロード

In [81]:
# Load from local file
from langchain.vectorstores import FAISS
vectorstore = FAISS.load_local("../data/db/faiss_index", embeddings, allow_dangerous_deserialization=True) # これはpickleによるデシリアライゼーションの安全性に関する警告であり、自作のDBなど信頼できるソースからのファイルであれば True で問題ありません
# create retriever
retriever = vectorstore.as_retriever()

## メモ：Pydantic基本構造

- BaseModel
    - データクラスを簡単に定義するためのPydantic基本クラス
    - 自動的な型チェックとバリデーション機能を提供

```
class User(BaseModel):
    name: str
    age: int
```

- Field
    - フィールドの詳細設定を行うためのクラス
    - バリデーションルール、デフォルト値、制約などを設定

```
class User(BaseModel):
    name: str = Field(..., min_length=2)  # nameフィールドは文字列で必須で最小２文字という意味。... は必須を表す
    age: int = Field(ge=0)     # ageフィールドは必須ではないが入れるなら0以上（ge: greater than or equal）
```

**使用例**
# 使用例
user1 = User(name="太郎", age=20)  # OK
user2 = User(name="太郎")  # エラー: ageは必須（defaultがないため）
user3 = User(name="太郎", age=-1)  # エラー: ageは0以上である必要がある

Pydanticの特徴：

1. 自動型変換：

    - age="20" → age=20 （文字列から整数に変換）
    - age=20.5 → age=20 （小数から整数に変換）
    - 
2. バリデーションエラー：

    - 整数に変換できない値（例：age="ねこ"）
    - 制約に違反する値（例：-1）


``` 
from typing import Literal # あまりみない型ヒントだけど、、、
```

Literal型では：
  - 整数、浮動小数点数
  - 文字列
  - 真偽値
これらの組み合わせのいずれも使用可能です。

## **質問ルーター**
質問ルーターは、クエリに対して、

- 内部インデックスを使用して回答すべきか
- ウェブ検索を通じて回答すべきか
  
を決定します。

トピックに基づいてプロンプトを変更してください。

In [82]:
# create question router

from typing import Literal
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI

# define a data class
class RouteQuery(BaseModel):
    """Route a user query to the most relevant datasource."""
    datasource: Literal["vectorstore", "web_search"] = Field(
        ...,
        description="Given a user question choose to route it to web search or a vectorstore.",
    ) # datasourceフィールドが必須で["vectorstore", "web_search"]のうちどちらかが入っていないとエラーと言う定義

# LLM with function call
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0, max_completion_tokens=4096)
structured_llm_router = llm.with_structured_output(RouteQuery, method="function_calling") # Response typeが RouteQuery クラスになる


In [83]:
# テスト実行
response = structured_llm_router.invoke("AIの最新トレンドについて教えて")

# 結果の確認
print(f"Response type: {type(response)}")
print(f"Response: {response}")

Response type: <class '__main__.RouteQuery'>
Response: datasource='web_search'


In [84]:
# Prompt (add only topics that are present in you vectorstore)
system = """You are an expert at routing a user question to either a vectorstore or web search.
The vectorstore contains information on the following topics:
- Government Initiatives for Startup Development

If the question is related to these topics, route it to the vectorstore. Otherwise, use web search."""
route_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)

question_router = route_prompt | structured_llm_router

In [85]:
eval_df['question'][0]

'オープンイノベーション促進税制において、スタートアップ企業の株式取得に対する税制優遇措置は、新規発行株式と発行済株式の場合でそれぞれどのように異なり、またこれらの株式取得に対する所得控除の適用条件は具体的に何か？'

In [86]:
question_router.invoke({"question": eval_df['question'][0]})

RouteQuery(datasource='vectorstore')

In [87]:
question_router.invoke({"question": "猫と犬の共通祖先の名前は"})

RouteQuery(datasource='web_search')

## **Document Grader**
ドキュメント評価者は、検索してきたドキュメントが与えられたクエリに本当に関連しているかどうかを評価します。

In [88]:
# create grader for doc retriever
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI

# define a data class
class GradeDocuments(BaseModel):
    binary_score: str = Field(
        description="Documents are relevant to the question, 'yes' or 'no'"
    )

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0, max_completion_tokens=4096)
structured_llm_grader = llm.with_structured_output(GradeDocuments, method="function_calling") # LLM の返す出力が GradeDocuments の形式に自動的にパースされます。


In [89]:
# Prompt for the grader
system = """You are a grader assessing relevance of a retrieved document to a user question. \n
    If the document contains keyword(s) or semantic meaning related to the question, grade it as relevant. \n
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question."""

# あなたは、検索されたドキュメントがユーザーの質問に関連しているかどうかを評価する採点者です。
# ドキュメントが質問に関連するキーワードや意味的な意味を含んでいる場合、関連していると評価します。
# ドキュメントが質問に関連しているかどうかを示すために、「はい」または「いいえ」の2値スコアを付けます。（本当にこの型で返ってくるかは保証がないので、後でチェックのために structured_llm_grader を付ける）

grade_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Retrieved document: \n\n {document} \n\n User question: {question}"),
    ]
)

retrieval_grader = grade_prompt | structured_llm_grader # プロンプトテンプレートと構造化出力対応の LLM をパイプライン的に連結しています。これにより、grade_prompt でフォーマットされた入力が structured_llm_grader に渡され、LLM の応答が GradeDocuments 型として出力される流れとなります。

In [90]:
# testing grader example 1
question = "猫と犬の共通祖先の名前は"
docs = retriever.get_relevant_documents(question)
print(retrieval_grader.invoke({"question": question, "document": docs}))

binary_score='no'


# RAG Chain

In [91]:
# create document chain
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate

template = """"
You are a helpful assistant that answers questions based on the following context.
Use the provided context to answer the question.
Context: {context}
Question: {question}
Answer:

"""

prompt = ChatPromptTemplate.from_template(template)
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0, max_completion_tokens=4096)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = prompt | llm | StrOutputParser()

In [92]:
# response
generation = rag_chain.invoke({"context": docs, "question": question})
generation

'The name of the common ancestor of cats and dogs is not provided in the given context.'

## **Hallucination Grader**
幻覚評価者は、その Answer が与えられた事実に基づいていたり、裏付けられているかどうかを確認する。
言い換えると　ハルシネーションが 'ない'、'ある' を 'yes' or 'no' で評価する

In [93]:
# create grader for hallucination
# define a data class
class GradeHallucinations(BaseModel):
    """Binary score for hallucination present in generation answer."""

    binary_score: str = Field(
        description="Answer is grounded in the facts, 'yes' or 'no'"
    )


# LLM with function call
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0, max_completion_tokens=4096)
structured_llm_grader = llm.with_structured_output(GradeHallucinations, method="function_calling")

# prompt for the grader
system = """You are a grader assessing whether an LLM generation is grounded in / supported by a set of retrieved facts. \n
     Give a binary score 'yes' or 'no'. 'Yes' means that the answer is grounded in / supported by the set of facts."""
hallucination_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Set of facts: \n\n {documents} \n\n LLM generation: {generation}"),
    ]
)

hallucination_grader = hallucination_prompt | structured_llm_grader
hallucination_grader.invoke({"documents": docs, "generation": generation})

GradeHallucinations(binary_score='no')

## **Answer Grader**
採点者は、回答が与えられた質問に効果的に対処しているかどうかを評価します。

In [94]:
# create grader for answer
# define a data class
class GradeAnswer(BaseModel):
    """Binary score to assess answer addresses question."""

    binary_score: str = Field(
        description="Answer addresses the question, 'yes' or 'no'"
    )


# LLM with function call
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0, max_completion_tokens=4096)
structured_llm_grader = llm.with_structured_output(GradeAnswer, method="function_calling")

# prompt for the grader
system = """You are a grader assessing whether an answer addresses / resolves a question \n
     Give a binary score 'yes' or 'no'. Yes' means that the answer resolves the question."""
answer_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "User question: \n\n {question} \n\n LLM generation: {generation}"),
    ]
)

answer_grader = answer_prompt | structured_llm_grader
answer_grader.invoke({"question": question, "generation": generation})

GradeAnswer(binary_score='no')

# Web Search

In [95]:
# define web search
from langchain_community.tools.tavily_search import TavilySearchResults
web_search_tool = TavilySearchResults(k=3)

## **Create Graph**

### **Define Graph State**

In [96]:
# define a data class for state
from typing import List, Optional
from typing_extensions import TypedDict

class GraphState(TypedDict, total=False):# total=Falseで、必須ではないフィールドも追加可能
    question: str
    generation: str
    documents: List[str]
    # 追加：評価結果や状態を格納するフィールド
    status: Optional[str]  # "useful", "not_useful", "max_retries_reached" などの状態
    retry_count: Optional[int]  # 再試行回数の追跡用

In [97]:
# define graph steps
from langchain.schema import Document
from pprint import pprint
import time 

MAX_RETRIES = 2

# all nodes return state dict
def retrieve(state):

    print("---関連文書の取得---")
    question = state["question"]

    # Retrieval
    documents = retriever.invoke(question)
    return {"documents": documents, "question": question}


def generate(state):

    print("---回答生成---")
    question = state["question"]
    documents = state["documents"]

    # RAG generation
    generation = rag_chain.invoke({"context": documents, "question": question})
    return {"documents": documents, "question": question, "generation": generation}


def grade_documents(state):

    print("---質問と検索文書に関連性があるかを確認---")
    question = state["question"]
    documents = state["documents"]

    # Score each doc
    filtered_docs = []
    for d in documents:
        score = retrieval_grader.invoke(
            {"question": question, "document": d.page_content}
        )
        grade = score.binary_score
        if grade == "yes":
            print("---評価: 質問と文書は関連しています---")
            filtered_docs.append(d)
        else:
            print("---評価: 質問と文書は関連していません---")
            continue
    return {"documents": filtered_docs, "question": question}


def web_search(state):

    print("---WEB SEARCH---")
    question = state["question"]

    # Web search
    docs = web_search_tool.invoke({"query": question})
    web_results = "\n".join([d["content"] for d in docs])
    web_results = Document(page_content=web_results)

    return {"documents": web_results, "question": question}


# edges　return strings
def route_question(state):

    print("---ROUTE QUESTION---")
    question = state["question"]
    source = question_router.invoke({"question": question})
    if source.datasource == "web_search":
        print("---ROUTE QUESTION TO WEB SEARCH---")
        return "web_search"
    elif source.datasource == "vectorstore":
        print("---ROUTE QUESTION TO RAG---")
        return "vectorstore"


def decide_to_generate(state):

    print("---証拠があれば回答を生成---")
    state["question"]
    filtered_documents = state["documents"]

    if not filtered_documents:
        # All documents have been filtered check_relevance
        # We will re-generate a new query
        print(
            "---決定：すべてのドキュメントは質問に関連していない。クエリを変換する。---"
        )
        return "transform_query"
    else:
        # We have relevant documents, so generate answer
        print("---決定：回答生成---")
        return "generate"


def grade_generation_v_documents_and_question(state):

    print("---CHECK HALLUCINATIONS---")
    question = state["question"]
    documents = state["documents"]
    generation = state["generation"]

    # 再試行回数を取得（初回は 0）
    retry_count = state.get("retry_count", 0)
    if retry_count >= MAX_RETRIES:
        state["status"] = "max_retries_reached"
        return state["status"]

    # 幻覚評価実施
    score = hallucination_grader.invoke({"documents": documents, "generation": generation})
    grade = score.binary_score

    # 幻覚チェック結果に応じた分岐
    if grade == "yes":
        print("---決定: ハルシネーションなし---")
        print("---生成結果の妥当性評価開始---")
        score = answer_grader.invoke({"question": question, "generation": generation})
        grade = score.binary_score
        if grade == "yes":
            print("---決定: 生成結果に妥当性あり---")
            state["status"] = "useful"
            return state["status"]
        else:
            print("---決定: 生成結果に妥当性なし---")
            state["retry_count"] = retry_count + 1
            state["status"] = "not useful"
            return state["status"]
    else:
        print("---決定: ハルシネーションあり、再試行します---")
        state["retry_count"] = retry_count + 1
        state["status"] = "not supported"
        return state["status"]

### **Build Graph**

In [98]:
# Build graph
from langgraph.graph import END, StateGraph, START

workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("web_search", web_search)  # web search
workflow.add_node("retrieve", retrieve)  # retrieve
workflow.add_node("grade_documents", grade_documents)  # grade documents
workflow.add_node("generate", generate)  # generatae
  # transform_query

# Build graph
workflow.add_conditional_edges(
    START,
    route_question,
    {
        "web_search": "web_search",
        "vectorstore": "retrieve",
    },
)
workflow.add_edge("web_search", "generate")
workflow.add_edge("retrieve", "grade_documents")
workflow.add_conditional_edges(
    "grade_documents",
    decide_to_generate,
    {
        "generate": "generate",
    },
)

workflow.add_conditional_edges(
    "generate",
    grade_generation_v_documents_and_question,
    {
        "not supported": "generate",
        "not useful": END,
        "max_retries_reached": END,
        "useful": END,
    },
)

# Compile
app = workflow.compile()
# print("```mermaid")
# print(app.get_graph().draw_mermaid())
# print("```")

# test app

In [99]:
# Final generation example 1 (web search)
from pprint import pprint

inputs = {"question": "猫と犬の共通祖先の名前は"}
for output in app.stream(inputs):
    for key, value in output.items():
        pprint(f"Node '{key}':")
    pprint("\n---\n")

pprint(value["generation"])

---ROUTE QUESTION---
---ROUTE QUESTION TO WEB SEARCH---
---WEB SEARCH---
"Node 'web_search':"
'\n---\n'
---回答生成---


BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 16385 tokens. However, you requested 17286 tokens (13190 in the messages, 4096 in the completion). Please reduce the length of the messages or completion.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}

In [ ]:
# Final generation example 2 (relevant documents)
inputs = {"question": eval_df['question'][0]}
for output in app.stream(inputs):
    for key, value in output.items():
        pprint(f"Node '{key}':")
    pprint("\n---\n")

pprint(value["generation"])

---ROUTE QUESTION---
---ROUTE QUESTION TO RAG---
---関連文書の取得---
"Node 'retrieve':"
'\n---\n'
---質問と検索文書に関連性があるかを確認---
---評価: 質問と文書は関連しています---
---評価: 質問と文書は関連していません---
---評価: 質問と文書は関連しています---
---評価: 質問と文書は関連していません---
---証拠があれば回答を生成---
---決定：回答生成---
"Node 'grade_documents':"
'\n---\n'
---回答生成---
---CHECK HALLUCINATIONS---
---決定: ハルシネーションなし---
---生成結果の妥当性評価開始---
---決定: 生成結果に妥当性あり---
"Node 'generate':"
'\n---\n'
('新規発行株式と発行済株式の取得に対する税制優遇措置は、取得額換算の上限額が異なります。新規発行株式の場合、上限額は1件あたり12.5億円（50億円/件まで）で、年間では125億円/社（500億円/社まで）です。一方、発行済株式の場合、上限額は1件あたり50億円（200億円/件まで）です。\n'
 '\n'
 '株式取得に対する所得控除の適用条件としては、取得株式の25％を所得控除することが可能です。ただし、M＆A型については、5年以内にスタートアップが成長投資・事業成長の要件を満たさなかった場合等には、所得控除分を一括取り戻す規定があります。\n'
 '\n'
 'また、株式取得の下限額も設定されており、大企業の場合は1億円/件、中小企業の場合は1千万円/件となっています。海外スタートアップの場合は、一律で5億円/件となっています。')


In [ ]:
# Create Create a a new new row ro
new_row = {
    'question': '猫と犬の共通祖先の名前は？',
    'target_answer': 'ミアキス',
    'target_page_no': None
}

# Add the new row to eval_df
eval_df = pd.concat([eval_df, pd.DataFrame([new_row])], ignore_index=True)
eval_df

,question,target_answer,target_page_no
0,オープンイノベーション促進税制において、スタートアップ企業の株式取得に対する税制優遇措置は、...,オープンイノベーション促進税制の下で、新規発行株式の取得は「新規出資型」として分類され、発行...,13
1,イノベーション拠点税制における所得控除について、控除対象となる研究開発活動に関して具体的にど...,イノベーション拠点税制における所得控除の対象となるためには、企業が主に「国内で」「自ら」開発...,16
2,「カーブアウト加速等支援事業」の主な目的は何ですか？,事業会社に蓄積されている技術を活用し、新たな会社を立ち上げた者や立ち上げる意思を持つ者に研究...,21
3,グローバル・アクセラレーション・ハブの拠点がある北米の都市を全て教えてください。,グローバル・アクセラレーション・ハブの北米の拠点は、ボストン、ニューヨーク、シカゴ、オーステ...,24
4,産業革新投資機構がベンチャー・グロース・インベストメンツを通じて設立したファンドについて、資...,産業革新投資機構(JIC)は子会社であるベンチャー・グロース・インベストメンツ(VGI)を通...,32
5,スタートアップ支援資金と挑戦支援資本強化特別貸付の融資限度額と返済期間の違いに加えて、要件や...,スタートアップ支援資金は融資限度額が20億円で直接貸付、返済期間は20年以内、要件はJVCA...,33
6,宇宙戦略基金の設立と関連する技術開発テーマの具体的な支援分野について説明し、各分野間でどのよ...,宇宙戦略基金は、民間企業や大学、スタートアップ、国立研究機関に対して10年間にわたる研究開発...,38
7,猫と犬の共通祖先の名前は？,ミアキス,None


In [ ]:
from pprint import pprint
import pandas as pd

outputs = []

# DataFrame の "question" 列をリスト化
inputs = eval_df["question"].tolist()

for i, question_text in enumerate(inputs):
    state_input = {"question": question_text}
    output = app.invoke(state_input)
    pprint(f"{output['question']=}")
    
#     # 結果となる状態情報は END キー、または generate キーに入っている前提
#     if output.get("END"):
#         value = output["END"]
#     elif output.get("generate"):
#         value = output["generate"]
#     else:
#         print(f"質問 {i} の出力に期待するキーが見つかりません。")
#         continue

#     # 生成結果が存在しない場合も状態情報を含める
    question = value.get("question", question_text)
    documents = value.get("documents", [])
    generation = value.get("generation", "")
    # final_status = value.get("status", "")
#     print(f"{documents=}")
    
#     # 各ドキュメントからページ内容を取得（Document 型なら page_content、タプルならその最初の要素を利用）
#     retrieved_contexts = []
#     if documents:
#         for doc in documents:
#             if hasattr(doc, "page_content"):
#                 retrieved_contexts.append(doc.page_content)
#             elif isinstance(doc, tuple) and len(doc) > 0:
#                 retrieved_contexts.append(doc[0])
#             else:
#                 retrieved_contexts.append(str(doc))
#     else:
#         retrieved_contexts = []

    reference = eval_df["target_answer"].iloc[i]

    outputs.append({
        "user_input": question,
        "retrieved_contexts": documents,
        "response": generation,
        "reference": reference,
        # "final_status": final_status,
    })

pprint(f"{outputs=}")

---ROUTE QUESTION---
---ROUTE QUESTION TO RAG---
---関連文書の取得---
---質問と検索文書に関連性があるかを確認---
---評価: 質問と文書は関連しています---
---評価: 質問と文書は関連しています---
---評価: 質問と文書は関連しています---
---評価: 質問と文書は関連していません---
---証拠があれば回答を生成---
---決定：回答生成---
---回答生成---
---CHECK HALLUCINATIONS---
---決定: ハルシネーションなし---
---生成結果の妥当性評価開始---
---決定: 生成結果に妥当性あり---
"output['question']='オープンイノベーション促進税制において、スタートアップ企業の株式取得に対する税制優遇措置は、新規発行株式と発行済株式の場合でそれぞれどのように異なり、またこれらの株式取得に対する所得控除の適用条件は具体的に何か？'"
---ROUTE QUESTION---
---ROUTE QUESTION TO RAG---
---関連文書の取得---
---質問と検索文書に関連性があるかを確認---
---評価: 質問と文書は関連しています---
---評価: 質問と文書は関連していません---
---評価: 質問と文書は関連していません---
---評価: 質問と文書は関連していません---
---証拠があれば回答を生成---
---決定：回答生成---
---回答生成---
---CHECK HALLUCINATIONS---
---決定: ハルシネーションなし---
---生成結果の妥当性評価開始---
---決定: 生成結果に妥当性あり---
"output['question']='イノベーション拠点税制における所得控除について、控除対象となる研究開発活動に関して具体的にどのような条件が求められますか？'"
---ROUTE QUESTION---
---ROUTE QUESTION TO RAG---
---関連文書の取得---
---質問と検索文書に関連性があるかを確認---
---評価: 質問と文書は関連しています---
---評価: 質問と文書は関連していません---
---評価: 質問と文書は関

BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 8192 tokens. However, your messages resulted in 13185 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}

In [ ]:
import pandas as pd
from pprint import pprint

df = pd.DataFrame(outputs)
df.head(3)

""


In [ ]:
pprint(df.to_dict(orient="records"))

In [ ]:
from ragas import evaluate
from ragas.llms import LangchainLLMWrapper
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness
# from ragas.metrics import Faithfulness
from ragas import EvaluationDataset

evaluation_dataset = EvaluationDataset.from_list(outputs)
evaluator_llm = LangchainLLMWrapper(llm)

result = evaluate(
    dataset=evaluation_dataset,
    metrics=[LLMContextRecall(), Faithfulness(), FactualCorrectness()],
    # metrics=[Faithfulness()],
    llm=evaluator_llm,
)

result